In [83]:
# Imports & quick checks
import os, sys, json, math, warnings, gc, time, random
from pathlib import Path
from IPython.display import display

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, roc_curve, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.calibration import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from scipy import stats

# Try XGBoost for meta
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except Exception as e:
    XGB_AVAILABLE = False
    print("xgboost not installed; meta-XGB will be skipped.")

# Try LightGBM
try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except Exception as e:
    LGB_AVAILABLE = False
    print("lightgbm not installed; LightGBM will be skipped.")

# Try CatBoost
try:
    import catboost as cb
    CB_AVAILABLE = True
except Exception as e:
    CB_AVAILABLE = False
    print("catboost not installed; CatBoost will be skipped.")

warnings.filterwarnings('ignore')
RANDOM_BASE = 42
np.random.seed(RANDOM_BASE)
random.seed(RANDOM_BASE)

ROOT = Path.cwd()
DATA_DIR = ROOT / 'Data'
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH = DATA_DIR / 'test.csv'
SAMPLE_SUB_PATH = DATA_DIR / 'sample_submission.csv'
SUB_DIR = ROOT / 'submissions'
SUB_DIR.mkdir(exist_ok=True, parents=True)

print(f'ROOT: {ROOT}')
print(f'Files exist? train={TRAIN_PATH.exists()} test={TEST_PATH.exists()} sample={SAMPLE_SUB_PATH.exists()}')

ROOT: /Users/lionelweng/Downloads/s5e11-Predicting-Loan-Payback
Files exist? train=True test=True sample=True


In [84]:
# Config & target/id detection
TARGET_CANDIDATES = ['target','TARGET','label','Label','default','is_default','loan_status','loan_repaid']
ID_CANDIDATES = ['id','ID','loan_id','Loan_ID']

def detect_columns(df: pd.DataFrame):
    cols = df.columns.tolist()
    id_col = None
    for c in ID_CANDIDATES:
        if c in cols:
            id_col = c
            break
    
    target_col = None
    for c in TARGET_CANDIDATES:
        if c in cols:
            target_col = c
            break
    if target_col is None:
        # Heuristic: last column if binary-like
        last = cols[-1]
        if df[last].dropna().isin([0,1]).mean() > 0.9:
            target_col = last
    return id_col, target_col

# Peek few rows to detect columns
preview = pd.read_csv(TRAIN_PATH, nrows=100)
ID_COL, TARGET = detect_columns(preview)
print('Detected ID_COL=', ID_COL, ' TARGET=', TARGET)
assert TARGET is not None, 'Target column not detected; please set TARGET manually.'

# Load full data
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH) if TEST_PATH.exists() else None
print(train.shape, 'train shape')
if test is not None:
    print(test.shape, 'test shape')

Detected ID_COL= id  TARGET= loan_paid_back
(593994, 13) train shape
(254569, 12) test shape
(593994, 13) train shape
(254569, 12) test shape


In [ ]:
≈# WINNING Feature Engineering Strategy (s5e11 reference)

y = train[TARGET].astype(int)
X = train.drop(columns=[TARGET] + ([ID_COL] if ID_COL else []))
X_test = None
if 'test' in globals() and test is not None:
    X_test = test.drop(columns=[ID_COL] if ID_COL else [])

num_cols_orig = X.select_dtypes(include=['number','float','int','Int8','Int16','Int32','Int64']).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols_orig]

print(f'Original: {len(num_cols_orig)} numeric, {len(cat_cols)} categorical')

# 1. LOAD ORIGINAL DATASET (The Stabilizer)
print('\n📚 Loading original reference dataset...')
orig = None
orig_paths = [
    DATA_DIR / 'loan_dataset_20000.csv',
    ROOT / 'loan_dataset_20000.csv',
    Path('/kaggle/input/loan-prediction-dataset-2025/loan_dataset_20000.csv')
]

for path in orig_paths:
    if path.exists():
        try:
            orig = pd.read_csv(path)
            print(f'  ✓ Loaded original dataset from: {path}')
            print(f'  Shape: {orig.shape}')
            break
        except Exception as e:
            print(f'  ⚠️  Failed to load {path}: {e}')

if orig is None:
    print('  ⚠️  Original dataset not found. Using train-based target encoding instead.')
    print('  Tip: Place loan_dataset_20000.csv in the Data/ folder for better stability.')
    use_orig = False
else:
    use_orig = True
    # Detect target column in original dataset
    orig_target = None
    for t in ['loan_paid_back', 'target', 'TARGET', 'loan_status', 'loan_repaid']:
        if t in orig.columns:
            orig_target = t
            break
    if orig_target is None:
        print('  ⚠️  Target column not found in original dataset')
        use_orig = False
    else:
        print(f'  ✓ Using target column: {orig_target}')

# 0. ORDINAL ENCODING for grade_subgrade (explicit rank mapping)
GRADE_SUBGRADE_MAP = {}
grades = ['A', 'B', 'C', 'D', 'E', 'F']
rank = 1
for grade in grades:
    for subgrade in range(1, 6):
        GRADE_SUBGRADE_MAP[f'{grade}{subgrade}'] = rank
        rank += 1

if 'grade_subgrade' in X.columns:
    X['grade_subgrade_ordinal'] = X['grade_subgrade'].map(GRADE_SUBGRADE_MAP).fillna(0).astype(int)
    if X_test is not None and 'grade_subgrade' in X_test.columns:
        X_test['grade_subgrade_ordinal'] = X_test['grade_subgrade'].map(GRADE_SUBGRADE_MAP).fillna(0).astype(int)
    print(f'Created grade_subgrade_ordinal: 1-30 mapping')

# 0b. Binary flag for unemployment
if 'employment_status' in X.columns:
    X['is_unemployed'] = (X['employment_status'] == 'Unemployed').astype(int)
    if X_test is not None and 'employment_status' in X_test.columns:
        X_test['is_unemployed'] = (X_test['employment_status'] == 'Unemployed').astype(int)
    print(f'Created is_unemployed binary flag')

# 1. TARGET ENCODING for categorical features (10-fold CV to prevent leakage) with smoothing
from sklearn.model_selection import KFold
TARGET_ENCODED = {}

global_mean = y.mean()
smoothing_k = 50  # smoothing strength per instructions

for cat in cat_cols[:]:  # Encode all categoricals
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    X[f'{cat}_target_enc'] = 0.0
    for tr_idx, va_idx in kf.split(X):
        # Compute smoothed stats on training fold only
        stats_fold = train.iloc[tr_idx].groupby(cat)[TARGET].agg(['mean', 'count'])
        stats_fold['smooth'] = (stats_fold['mean'] * stats_fold['count'] + global_mean * smoothing_k) / (stats_fold['count'] + smoothing_k)
        mapping_fold = stats_fold['smooth']
        X.loc[X.index[va_idx], f'{cat}_target_enc'] = X.iloc[va_idx][cat].map(mapping_fold).fillna(global_mean)
    # For test, use full train stats (smoothed)
    if X_test is not None:
        stats_full = train.groupby(cat)[TARGET].agg(['mean', 'count'])
        stats_full['smooth'] = (stats_full['mean'] * stats_full['count'] + global_mean * smoothing_k) / (stats_full['count'] + smoothing_k)
        mapping_full = stats_full['smooth']
        X_test[f'{cat}_target_enc'] = X_test[cat].map(mapping_full).fillna(global_mean)
    TARGET_ENCODED[cat] = f'{cat}_target_enc'
    print(f'Target encoded (smoothed): {cat}')

# 2. LOG TRANSFORMS for skewed features (address heavy skew)
LOG_COLS = ['annual_income', 'loan_amount', 'debt_to_income_ratio']
for col in LOG_COLS:
    if col in num_cols_orig:
        X[f'{col}_log'] = np.log1p(X[col].clip(lower=0))
        if X_test is not None:
            X_test[f'{col}_log'] = np.log1p(X_test[col].clip(lower=0))
        print(f'Created log transform: {col}_log')

# 2b. Log income to log loan ratio
if 'annual_income' in num_cols_orig and 'loan_amount' in num_cols_orig:
    X['log_income_to_log_loan'] = np.log1p(X['annual_income'].clip(lower=0)) / (np.log1p(X['loan_amount'].clip(lower=0)) + 1e-6)
    if X_test is not None:
        X_test['log_income_to_log_loan'] = np.log1p(X_test['annual_income'].clip(lower=0)) / (np.log1p(X_test['loan_amount'].clip(lower=0)) + 1e-6)
    print('Created log_income_to_log_loan ratio')

# 3. INTERACTION FEATURES (ratios + products + polynomials)
IMPORTANT_PAIRS = [
    ('loan_amount', 'annual_income'),
    ('loan_amount', 'credit_score'),
    ('debt_to_income_ratio', 'credit_score'),
    ('annual_income', 'credit_score'),
    ('interest_rate', 'loan_amount'),
]

for c1, c2 in IMPORTANT_PAIRS:
    if c1 in num_cols_orig and c2 in num_cols_orig:
        # Ratio
        X[f'{c1}_div_{c2}'] = X[c1] / (X[c2] + 1e-6)
        if X_test is not None:
            X_test[f'{c1}_div_{c2}'] = X_test[c1] / (X_test[c2] + 1e-6)
        # Product
        X[f'{c1}_x_{c2}'] = X[c1] * X[c2]
        if X_test is not None:
            X_test[f'{c1}_x_{c2}'] = X_test[c1] * X_test[c2]
        # Difference
        X[f'{c1}_minus_{c2}'] = X[c1] - X[c2]
        if X_test is not None:
            X_test[f'{c1}_minus_{c2}'] = X_test[c1] - X_test[c2]

# Risk Ratio: Interest Rate relative to Credit Score (High Rate + Low Score = Extreme Risk)
if 'interest_rate' in num_cols_orig and 'credit_score' in num_cols_orig:
    X['risk_ratio'] = X['interest_rate'] / (X['credit_score'] + 1e-6)
    if X_test is not None:
        X_test['risk_ratio'] = X_test['interest_rate'] / (X_test['credit_score'] + 1e-6)
    print('Created risk_ratio feature (interest_rate / credit_score)')

# Estimated Monthly Payment / Monthly Income (Payment-to-Income proxy)
if 'loan_amount' in num_cols_orig and 'interest_rate' in num_cols_orig and 'annual_income' in num_cols_orig:
    X['pti_proxy'] = (X['loan_amount'] * X['interest_rate']) / (X['annual_income'] + 1)
    if X_test is not None:
        X_test['pti_proxy'] = (X_test['loan_amount'] * X_test['interest_rate']) / (X_test['annual_income'] + 1)
    print('Created pti_proxy feature ((loan_amount * interest_rate) / annual_income)')

# 4. POLYNOMIAL FEATURES (square key predictors)
for col in ['credit_score', 'annual_income', 'loan_amount'][:]:
    if col in num_cols_orig:
        X[f'{col}_squared'] = X[col] ** 2
        X[f'{col}_sqrt'] = np.sqrt(X[col].clip(lower=0))
        if X_test is not None:
            X_test[f'{col}_squared'] = X_test[col] ** 2
            X_test[f'{col}_sqrt'] = np.sqrt(X_test[col].clip(lower=0))

# 5. BINNING FEATURES (discretize continuous)
for col in ['credit_score', 'annual_income', 'loan_amount'][:]:
    if col in num_cols_orig:
        X[f'{col}_bin'] = pd.qcut(X[col], q=10, labels=False, duplicates='drop')
        if X_test is not None:
            # Use train quantiles for test
            quantiles = X[col].quantile(np.linspace(0, 1, 11)).unique()
            X_test[f'{col}_bin'] = pd.cut(X_test[col], bins=quantiles, labels=False, include_lowest=True).fillna(5)

num_cols = X.select_dtypes(include=['number','float','int','Int8','Int16','Int32','Int64']).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]
print(f'Engineered: {len(num_cols)} numeric, {len(cat_cols)} categorical')
print(f'Feature count: {X.shape[1]} (from {len(num_cols_orig) + len(cat_cols)})')

# Global fill for Sklearn models
X = X.fillna(0)
if X_test is not None:
    X_test = X_test.fillna(0)

# 6. KNN INJECTION: Local manifold structure (Magic Feature for loan prediction)
print('\n🎯 KNN Feature Engineering (Orthogonal Diversity)')
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Select numeric features for KNN (exclude categorical and target-encoded)
knn_feature_cols = [c for c in num_cols if '_target_enc' not in c and '_bin' not in c]
print(f'  Using {len(knn_feature_cols)} numeric features for KNN')

# Normalize data
scaler = StandardScaler()
X_knn_scaled = scaler.fit_transform(X[knn_feature_cols])
if X_test is not None:
    X_test_knn_scaled = scaler.transform(X_test[knn_feature_cols])

# Fit KNN on training data
knn = NearestNeighbors(n_neighbors=11, n_jobs=-1)  # 11 to exclude self
knn.fit(X_knn_scaled)

# Get neighbors and distances for train
distances_train, indices_train = knn.kneighbors(X_knn_scaled)
# Exclude self (first neighbor) and take mean of remaining 10
knn_risk_train = np.array([y.iloc[indices_train[i, 1:]].mean() for i in range(len(indices_train))])
X['knn_risk'] = knn_risk_train
X['knn_dist'] = distances_train[:, 1:].mean(axis=1)

if X_test is not None:
    # Get neighbors and distances for test
    distances_test, indices_test = knn.kneighbors(X_test_knn_scaled)
    # For test, use all 10 neighbors (no self to exclude)
    knn_risk_test = np.array([y.iloc[indices_test[i, :10]].mean() for i in range(len(indices_test))])
    X_test['knn_risk'] = knn_risk_test
    X_test['knn_dist'] = distances_test[:, :10].mean(axis=1)

print(f'  ✓ Created knn_risk (avg target of 10 neighbors) and knn_dist (avg distance)')
print(f'  knn_risk range: [{X["knn_risk"].min():.3f}, {X["knn_risk"].max():.3f}]')

# Update numeric columns list
num_cols = X.select_dtypes(include=['number','float','int','Int8','Int16','Int32','Int64']).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]
print(f'After KNN: {len(num_cols)} numeric, {len(cat_cols)} categorical')

# Simplified preprocessing
numeric_tf = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])
categorical_tf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocess = ColumnTransformer(transformers=[
    ('num', numeric_tf, num_cols),
    ('cat', categorical_tf, cat_cols)
])

def build_model(name: str):
    if name == 'xgb' and XGB_AVAILABLE:
        return 'xgb_raw'
    elif name == 'lgb' and LGB_AVAILABLE:
        return 'lgb_raw'
    elif name == 'cb' and CB_AVAILABLE:
        return 'cb_raw'
    else:
        raise ValueError(f'Model {name} not available')


Original: 5 numeric, 6 categorical
Created grade_subgrade_ordinal: 1-30 mapping
Created is_unemployed binary flag
Target encoded (smoothed): gender
Target encoded (smoothed): gender
Target encoded (smoothed): marital_status
Target encoded (smoothed): marital_status
Target encoded (smoothed): education_level
Target encoded (smoothed): education_level
Target encoded (smoothed): employment_status
Target encoded (smoothed): employment_status
Target encoded (smoothed): loan_purpose
Target encoded (smoothed): loan_purpose
Target encoded (smoothed): grade_subgrade
Created log transform: annual_income_log
Created log transform: loan_amount_log
Created log transform: debt_to_income_ratio_log
Created log_income_to_log_loan ratio
Created risk_ratio feature (interest_rate / credit_score)
Created pti_proxy feature ((loan_amount * interest_rate) / annual_income)
Target encoded (smoothed): grade_subgrade
Created log transform: annual_income_log
Created log transform: loan_amount_log
Created log trans

In [86]:
# CV, metrics, threshold sweep, and isotonic calibration utils
def get_cv(n_splits=5, seed=42):
    return StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

def threshold_sweep(y_true, prob, thresholds=None):
    if thresholds is None:
        thresholds = np.linspace(0.05, 0.95, 19)
    best = {'threshold': None, 'f1': -1, 'precision': None, 'recall': None}
    for t in thresholds:
        pred = (prob >= t).astype(int)
        f1 = f1_score(y_true, pred)
        if f1 > best['f1']:
            # compute precision & recall via confusion matrix
            tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
            prec = tp / (tp + fp + 1e-9)
            rec = tp / (tp + fn + 1e-9)
            best = {'threshold': float(t), 'f1': float(f1), 'precision': float(prec), 'recall': float(rec)}
    return best

def fit_isotonic(y_true, prob):
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(prob, y_true)
    return iso

In [87]:
# 📊 Performance tracking utilities

def print_progress_bar(current, target=0.93, width=50):
    """Visual progress bar toward 93% AUC"""
    min_val = 0.90
    max_val = 0.94
    progress = (current - min_val) / (max_val - min_val)
    filled = int(width * progress)
    bar = '█' * filled + '░' * (width - filled)
    pct = current * 100
    target_pct = target * 100
    
    print(f'\n📊 Progress to {target_pct:.1f}%:')
    print(f'[{bar}] {pct:.3f}%')
    
    if current >= target:
        print('🎉 TARGET ACHIEVED! 🎉')
    else:
        gap = (target - current) * 100
        print(f'Gap: {gap:.3f} pp')

def compare_techniques(base_auc, l2_auc, l3_auc, cal_auc):
    """Show incremental gains from each technique"""
    print(f'\n📈 TECHNIQUE BREAKDOWN:')
    print(f'   Base (L1):       {base_auc:.5f}')
    print(f'   + L2 Meta:       {l2_auc:.5f}  (+{(l2_auc-base_auc)*100:.3f} pp)')
    print(f'   + L3 Pseudo:     {l3_auc:.5f}  (+{(l3_auc-l2_auc)*100:.3f} pp)')
    print(f'   + Calibration:   {cal_auc:.5f}  (+{(cal_auc-l3_auc)*100:.3f} pp)')
    print(f'   ─────────────────────────────────────')
    print(f'   TOTAL GAIN:      +{(cal_auc-base_auc)*100:.3f} pp')
    
print('Progress tracking ready ✓')

Progress tracking ready ✓


In [88]:
# L1 Base Models: EXTREME tuning for 93%+ (3 diverse boosters + MLP for orthogonal diversity)

def train_base_models(X, y, X_test=None, seed=42, n_splits=5):
    from sklearn.preprocessing import StandardScaler
    from sklearn.neural_network import MLPClassifier
    
    cv = get_cv(n_splits=n_splits, seed=seed)
    
    # Prepare scaled data for MLP (neural nets need normalized inputs)
    num_cols_for_mlp = X.select_dtypes(include=['number','float','int','Int8','Int16','Int32','Int64']).columns.tolist()
    scaler_mlp = StandardScaler()
    X_scaled = scaler_mlp.fit_transform(X[num_cols_for_mlp])
    X_test_scaled = None
    if X_test is not None:
        X_test_scaled = scaler_mlp.transform(X_test[num_cols_for_mlp])
    
    # Use ALL available gradient boosters
    base_models_config = []
    
    if XGB_AVAILABLE:
        base_models_config.append(('xgb', {
            'n_estimators': 1500, 'learning_rate': 0.015, 'max_depth': 8,
            'subsample': 0.8, 'colsample_bytree': 0.8, 
            'reg_lambda': 3.0, 'reg_alpha': 0.8, 'min_child_weight': 2,
            'tree_method': 'hist', 'n_jobs': -1,
            'enable_categorical': True,
        }))
    
    if LGB_AVAILABLE:
        base_models_config.append(('lgb', {
            'n_estimators': 1500, 'learning_rate': 0.015, 'max_depth': 10, 'num_leaves': 255,
            'subsample': 0.8, 'colsample_bytree': 0.8, 
            'reg_lambda': 3.0, 'reg_alpha': 0.6, 'min_child_samples': 20,
            'verbose': -1, 'n_jobs': -1, 'force_col_wise': True
        }))
    
    if CB_AVAILABLE:
        base_models_config.append(('cb', {
            'iterations': 1500, 'learning_rate': 0.015, 'depth': 8,
            'l2_leaf_reg': 5, 'border_count': 254, 'min_data_in_leaf': 10,
            'verbose': 0, 'thread_count': -1
        }))

    # Add Random Forest (Bagging) for diversity
    base_models_config.append(('rf', {
        'n_estimators': 300,
        'max_depth': 15,
        'min_samples_leaf': 5,
        'max_features': 'sqrt',
        'n_jobs': -1,
        'random_state': seed
    }))
    
    # Add MLP (Neural Network) for ORTHOGONAL DIVERSITY (smooth decision boundary)
    base_models_config.append(('mlp', {
        'hidden_layer_sizes': (256, 128),
        'early_stopping': True,
        'max_iter': 300,
        'random_state': seed,
        'verbose': False
    }))
    
    if not base_models_config:
        raise ValueError('No gradient boosters available! Install XGBoost, LightGBM, or CatBoost.')
    
    base_names = [name for name, _ in base_models_config]
    print(f'🚀 Training {len(base_names)} L1 base models: {base_names}')
    
    oof = np.zeros((len(X), len(base_names)))
    test_preds = np.zeros((len(X_test), len(base_names))) if X_test is not None else None
    aucs = {name: [] for name in base_names}

    for j, (name, params) in enumerate(base_models_config):
        fold_idx = 0
        for tr_idx, va_idx in cv.split(X, y):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            if name == 'xgb':
                # Cast object/bin columns to categorical for XGBoost
                cat_features_xgb = [c for c in X.columns if X[c].dtype == 'object' or '_bin' in c]
                X_tr_local = X_tr.copy()
                X_va_local = X_va.copy()
                for c in cat_features_xgb:
                    if c in X_tr_local.columns:
                        X_tr_local[c] = X_tr_local[c].astype('category')
                    if c in X_va_local.columns:
                        X_va_local[c] = X_va_local[c].astype('category')
                model = xgb.XGBClassifier(random_state=seed+fold_idx, **params)
                model.fit(X_tr_local, y_tr)
                p = model.predict_proba(X_va_local)[:,1]
            elif name == 'lgb':
                # LightGBM requires category dtype for categorical features (not object)
                cat_obj_cols = [c for c in X.columns if X[c].dtype == 'object']
                X_tr_local = X_tr.copy()
                X_va_local = X_va.copy()
                for c in cat_obj_cols:
                    if c in X_tr_local.columns:
                        X_tr_local[c] = X_tr_local[c].astype('category')
                    if c in X_va_local.columns:
                        X_va_local[c] = X_va_local[c].astype('category')
                model = lgb.LGBMClassifier(random_state=seed+fold_idx, **params)
                model.fit(X_tr_local, y_tr, categorical_feature=cat_obj_cols if cat_obj_cols else 'auto')
                p = model.predict_proba(X_va_local)[:,1]
            elif name == 'cb':
                # CatBoost requires str/int categorical features (not float/numeric bins)
                cat_features_cb = [c for c in X.columns if X[c].dtype == 'object' or '_bin' in c]
                X_tr_local = X_tr.copy()
                X_va_local = X_va.copy()
                # Convert bin columns (float) to string for CatBoost
                for c in cat_features_cb:
                    if '_bin' in c and c in X_tr_local.columns:
                        X_tr_local[c] = X_tr_local[c].fillna(-1).astype(int).astype(str)
                    if '_bin' in c and c in X_va_local.columns:
                        X_va_local[c] = X_va_local[c].fillna(-1).astype(int).astype(str)
                model = cb.CatBoostClassifier(
                    random_seed=seed+fold_idx, 
                    cat_features=cat_features_cb if cat_features_cb else None,
                    **params
                )
                model.fit(X_tr_local, y_tr)
                p = model.predict_proba(X_va_local)[:,1]
            elif name == 'rf':
                # RandomForest on numeric features only
                num_cols_rf = X.select_dtypes(include=['number','float','int','Int8','Int16','Int32','Int64']).columns.tolist()
                X_tr_local = X_tr[num_cols_rf]
                X_va_local = X_va[num_cols_rf]
                model = RandomForestClassifier(**params)
                model.fit(X_tr_local, y_tr)
                p = model.predict_proba(X_va_local)[:,1]
            elif name == 'mlp':
                # MLP on SCALED numeric features (critical for neural networks)
                X_tr_local = X_scaled[tr_idx]
                X_va_local = X_scaled[va_idx]
                model = MLPClassifier(**params)
                model.fit(X_tr_local, y_tr)
                p = model.predict_proba(X_va_local)[:,1]
            
            oof[va_idx, j] = p
            auc = roc_auc_score(y_va, p)
            aucs[name].append(auc)
            
            if X_test is not None:
                if name == 'xgb':
                    X_test_local = X_test.copy()
                    for c in cat_features_xgb:
                        if c in X_test_local.columns:
                            X_test_local[c] = X_test_local[c].astype('category')
                    test_preds[:, j] += model.predict_proba(X_test_local)[:,1] / cv.get_n_splits()
                elif name == 'lgb':
                    X_test_local = X_test.copy()
                    for c in cat_obj_cols:
                        if c in X_test_local.columns:
                            X_test_local[c] = X_test_local[c].astype('category')
                    test_preds[:, j] += model.predict_proba(X_test_local)[:,1] / cv.get_n_splits()
                elif name == 'cb':
                    X_test_local = X_test.copy()
                    for c in cat_features_cb:
                        if '_bin' in c and c in X_test_local.columns:
                            X_test_local[c] = X_test_local[c].fillna(-1).astype(int).astype(str)
                    test_preds[:, j] += model.predict_proba(X_test_local)[:,1] / cv.get_n_splits()
                elif name == 'rf':
                    num_cols_rf = X.select_dtypes(include=['number','float','int','Int8','Int16','Int32','Int64']).columns.tolist()
                    X_test_local = X_test[num_cols_rf]
                    test_preds[:, j] += model.predict_proba(X_test_local)[:,1] / cv.get_n_splits()
                elif name == 'mlp':
                    # MLP on scaled test data
                    test_preds[:, j] += model.predict_proba(X_test_scaled)[:,1] / cv.get_n_splits()
            fold_idx += 1
        
        mean_auc = np.mean(aucs[name])
        print(f"  ✓ {name}: {np.round(aucs[name], 5)} → mean {mean_auc:.5f}")
    
    return oof, test_preds, aucs


In [89]:
# Leakage audit utilities
# 1. Single-feature AUC to flag suspicious leak features.
# 2. Temporal leakage heuristic: columns that look like aggregates (e.g., total_*, avg_*) might encode future info.
# 3. Type casting helpers.

import re

LEAK_MAX_FEATURES = 40  # cap evaluation for speed

def single_feature_auc_scan(df: pd.DataFrame, y: pd.Series, max_features=LEAK_MAX_FEATURES):
    aucs = []
    for col in df.columns[:max_features]:
        try:
            if df[col].nunique() < 2:
                continue
            vals = df[col].fillna(df[col].median() if df[col].dtype != 'O' else 'missing')
            # For categorical -> encode label frequency
            if vals.dtype == 'O':
                mapping = vals.value_counts(normalize=True).to_dict()
                enc = vals.map(mapping).astype(float)
            else:
                enc = vals.astype(float)
            score = roc_auc_score(y, enc) if len(np.unique(enc)) > 1 else 0.5
            aucs.append((col, score))
        except Exception:
            continue
    aucs.sort(key=lambda x: x[1], reverse=True)
    return aucs

AGG_PATTERNS = [r'^total_', r'^sum_', r'^avg_', r'^mean_', r'^max_', r'^min_']

def looks_leaky(colname: str) -> bool:
    for pat in AGG_PATTERNS:
        if re.search(pat, colname):
            return True
    return False

# KS & PSI drift checks between train/test

def ks_stat(train_col, test_col):
    # dropna
    a = pd.Series(train_col).dropna()
    b = pd.Series(test_col).dropna()
    if a.nunique() < 2 or b.nunique() < 2:
        return 0.0
    try:
        stat, pval = stats.ks_2samp(a, b)
        return stat
    except Exception:
        return 0.0

# Population Stability Index for binned values

def psi(train_col, test_col, buckets=10):
    a = pd.Series(train_col).dropna()
    b = pd.Series(test_col).dropna()
    if a.nunique() < 2 or b.nunique() < 2:
        return 0.0
    quantiles = np.linspace(0, 1, buckets + 1)
    cuts = a.quantile(quantiles).unique()
    a_bins = pd.cut(a, bins=np.unique(cuts), include_lowest=True)
    b_bins = pd.cut(b, bins=np.unique(cuts), include_lowest=True)
    a_dist = a_bins.value_counts(normalize=True)
    b_dist = b_bins.value_counts(normalize=True)
    psi_val = 0.0
    for idx in a_dist.index:
        expected = a_dist.get(idx, 1e-6)
        actual = b_dist.get(idx, 1e-6)
        if expected > 0 and actual > 0:
            psi_val += (actual - expected) * math.log(actual / expected)
    return psi_val

DRIFT_REPORT_LIMIT = 40

def drift_report(train_df: pd.DataFrame, test_df: pd.DataFrame):
    rows = []
    shared = [c for c in train_df.columns if c in test_df.columns]
    for col in shared[:DRIFT_REPORT_LIMIT]:
        try:
            k = ks_stat(train_df[col], test_df[col])
            p = psi(train_df[col], test_df[col])
            rows.append({'feature': col, 'ks': k, 'psi': p})
        except Exception:
            continue
    rep = pd.DataFrame(rows)
    if not rep.empty:
        rep.sort_values(['ks','psi'], ascending=False, inplace=True)
    return rep

BOOL_LIKE = ['y','n','yes','no','true','false']

def cast_types(df: pd.DataFrame):
    for c in df.columns:
        if df[c].dtype == 'O':
            # bool-like
            low = df[c].str.lower()
            if low.isin(BOOL_LIKE).mean() > 0.9:
                df[c] = low.map({'y':1,'yes':1,'true':1,'n':0,'no':0,'false':0}).astype('Int8')
    return df

print('Leakage & drift utilities ready.')

Leakage & drift utilities ready.


In [90]:
# Apply type casting, leakage audit, and drift checks
# Must run after data load (Cell 3)

assert 'train' in globals(), 'Run the data load cell first.'

# 1) Type casting
if 'ID_COL' in globals() and ID_COL:
    train[ID_COL] = train[ID_COL].astype(str)
    if 'test' in globals() and test is not None and ID_COL in test.columns:
        test[ID_COL] = test[ID_COL].astype(str)

train = cast_types(train)
if 'test' in globals() and test is not None:
    test = cast_types(test)

# 2) Leakage audit (simple, top-N features)
feat_cols = [c for c in train.columns if c not in [TARGET] + ([ID_COL] if ID_COL else [])]
scan_df = train[feat_cols].copy()
scan_aucs = single_feature_auc_scan(scan_df, train[TARGET], max_features=min(LEAK_MAX_FEATURES, len(feat_cols)))
leaky = [c for (c, auc) in scan_aucs if auc >= 0.92 or auc <= 0.08 or looks_leaky(c)]

if len(leaky) > 0:
    print('Dropping suspicious leakage features:', leaky)
    train.drop(columns=[c for c in leaky if c in train.columns], inplace=True)
    if 'test' in globals() and test is not None:
        test.drop(columns=[c for c in leaky if c in test.columns], inplace=True)
else:
    print('No leakage features flagged by simple scan.')

# 3) Drift check (requires test)
if 'test' in globals() and test is not None:
    tr_common = train.drop(columns=[TARGET] + ([ID_COL] if ID_COL else []), errors='ignore')
    te_common = test.drop(columns=[ID_COL] if ID_COL else [], errors='ignore')
    rep = drift_report(tr_common, te_common)
    display(rep.head(12))
    # Drop worst offenders by relaxed rule
    drop_drift = rep[(rep['ks'] >= 0.3) | (rep['psi'] >= 0.3)]['feature'].tolist()
    if drop_drift:
        print('Dropping drift-heavy features:', drop_drift)
        train.drop(columns=[c for c in drop_drift if c in train.columns], inplace=True)
        test.drop(columns=[c for c in drop_drift if c in test.columns], inplace=True)
else:
    print('Test set not available; skipping drift check.')

print('Preprocessing audits complete.')

No leakage features flagged by simple scan.


,feature,ks,psi
4,interest_rate,0.002596,0.000062
1,debt_to_income_ratio,0.002063,0.000047
0,annual_income,0.001902,0.000039
2,credit_score,0.001877,0.000026
3,loan_amount,0.001703,0.000044


Preprocessing audits complete.


In [91]:
# L2 Meta: HILL CLIMBING optimizer for optimal ensemble weights (replaces XGBoost)

def train_meta_l2(oof_feats, y, test_feats=None, seed=42, X_orig=None, X_test_orig=None):
    """L2 Hill Climbing Meta-Learner: Find optimal weights to maximize AUC"""
    from scipy.optimize import minimize
    
    print(f'  📊 L2 Hill Climbing on {oof_feats.shape[1]} base model predictions')
    
    # Objective function: Minimize negative AUC (to maximize AUC)
    def objective(weights):
        # Weighted average of base model predictions
        weighted_pred = np.dot(oof_feats, weights)
        return -roc_auc_score(y, weighted_pred)
    
    # Constraints: weights must sum to 1
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    
    # Bounds: all weights must be non-negative
    bounds = [(0, 1) for _ in range(oof_feats.shape[1])]
    
    # Initial guess: equal weights
    initial_weights = np.ones(oof_feats.shape[1]) / oof_feats.shape[1]
    
    # Run optimization
    result = minimize(
        objective,
        initial_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000, 'ftol': 1e-9}
    )
    
    if result.success:
        optimal_weights = result.x
        oof_meta = np.dot(oof_feats, optimal_weights)
        meta_auc = roc_auc_score(y, oof_meta)
        
        print(f'  ✓ Hill Climbing optimized weights: {np.round(optimal_weights, 4)}')
        print(f'  ✓ L2 Hill Climbing Meta AUC: {meta_auc:.5f}')
        
        # Apply same weights to test predictions
        test_meta = None
        if test_feats is not None:
            test_meta = np.dot(test_feats, optimal_weights)
        
        return oof_meta, test_meta
    else:
        print(f'  ⚠️  Optimization failed, using equal weights')
        # Fallback to equal weights
        oof_meta = oof_feats.mean(axis=1)
        test_meta = test_feats.mean(axis=1) if test_feats is not None else None
        meta_auc = roc_auc_score(y, oof_meta)
        print(f'  ✓ L2 Equal Weights Meta AUC: {meta_auc:.5f}')
        return oof_meta, test_meta


def train_meta_l3_with_pseudo(oof_l2, y, test_l2, X, X_test, seed=42):
    """L3 Meta + PSEUDO-LABELING for final push to 93%+"""
    
    # PSEUDO-LABELING: Use high-confidence test predictions
    if test_l2 is not None and X_test is not None:
        # Select high-confidence test samples (≥0.90 or ≤0.10)
        high_conf_mask = (test_l2 > 0.90) | (test_l2 < 0.10)
        pseudo_labels = (test_l2 > 0.5).astype(int)
        
        n_pseudo = high_conf_mask.sum()
        if n_pseudo > 0:
            print(f'  🎭 Pseudo-labeling: {n_pseudo} high-confidence test samples')
            
            # Combine train + pseudo-labeled test
            X_combined = pd.concat([X, X_test.iloc[high_conf_mask]], axis=0, ignore_index=True)
            y_combined = pd.concat([y, pd.Series(pseudo_labels[high_conf_mask])], axis=0, ignore_index=True)
            oof_combined = np.concatenate([oof_l2, test_l2[high_conf_mask]])
            
            # Retrain L3 on combined data
            cv = get_cv(n_splits=5, seed=seed)
            oof_l3 = np.zeros(len(oof_combined))
            
            for fold, (tr_idx, va_idx) in enumerate(cv.split(oof_combined, y_combined)):
                X_tr, X_va = oof_combined[tr_idx].reshape(-1, 1), oof_combined[va_idx].reshape(-1, 1)
                y_tr, y_va = y_combined.iloc[tr_idx], y_combined.iloc[va_idx]
                
                clf = LogisticRegression(max_iter=5000, C=0.5)
                clf.fit(X_tr, y_tr)
                oof_l3[va_idx] = clf.predict_proba(X_va)[:,1]
            
            # Extract only original train predictions
            oof_l3_train = oof_l3[:len(y)]
            auc_l3 = roc_auc_score(y, oof_l3_train)
            print(f'  ✓ L3 Meta + Pseudo AUC: {auc_l3:.5f}')
            
            return oof_l3_train
    
    # Fallback: simple L3 without pseudo-labeling
    return oof_l2


In [92]:
# Training orchestrator: EXTREME pipeline for 93%+

def run_training_extreme(seeds=[42, 43], target_auc=0.93):
    """
    Multi-level stacking + pseudo-labeling pipeline
    L1 (base) → L2 (fast linear) → L3 (pseudo) → Calibration
    """
    results = []
    best = None
    
    for i, seed in enumerate(seeds):
        print(f"\n{'='*70}\n🚀 SEED {seed} ({i+1}/{len(seeds)}) — Targeting 93%+ AUC\n{'='*70}")
        
        # L1: Base models (XGB + LGB + CB)
        print('\n[L1] Training base models...')
        oof_l1, test_l1, base_aucs = train_base_models(X, y, X_test, seed=seed, n_splits=7)
        
        # L2: Fast Linear Meta model with grade ordinal
        print('\n[L2] Training fast linear meta...')
        oof_l2, test_l2 = train_meta_l2(oof_l1, y, test_l1, seed=seed, X_orig=X, X_test_orig=X_test)
        
        # L3: Pseudo-labeling (if available)
        print('\n[L3] Pseudo-labeling...')
        oof_l3 = train_meta_l3_with_pseudo(oof_l2, y, test_l2, X, X_test, seed=seed)
        
        # Isotonic calibration
        print('\n[CAL] Calibrating predictions...')
        iso = fit_isotonic(y.values, oof_l3)
        oof_cal = iso.predict(oof_l3)
        auc_cal = roc_auc_score(y, oof_cal)
        test_cal = iso.predict(test_l2) if test_l2 is not None else None
        
        # Threshold optimization
        best_thr = threshold_sweep(y.values, oof_cal)
        
        print(f'\n{"="*70}')
        print(f'🎯 FINAL AUC (calibrated): {auc_cal:.5f}')
        print(f'📊 Best threshold: {best_thr["threshold"]:.3f} (F1={best_thr["f1"]:.4f})')
        print(f'{"="*70}')
        
        record = {
            'seed': seed,
            'auc_l2': roc_auc_score(y, oof_l2),
            'auc_l3': roc_auc_score(y, oof_l3),
            'auc_cal': auc_cal,
            'best_thr': best_thr,
        }
        results.append(record)
        
        if (best is None) or (auc_cal > best['auc_cal']):
            best = {
                **record,
                'oof_cal': oof_cal,
                'test_cal': test_cal,
                'base_aucs': base_aucs
            }
            print(f'✨ NEW BEST: {auc_cal:.5f}')
        
        if auc_cal >= target_auc:
            print(f'\n🏆 BREAKTHROUGH! Hit {target_auc:.1%} target!')
            break
    
    return pd.DataFrame(results), best

In [93]:
# 🚀 EXECUTE: Extreme training for 93%+ AUC

print('🎯 TARGET: Break 93% AUC barrier')
print('📈 Strategy: L1→L2→L3 stacking + pseudo-labeling + calibration')
print('⏱️  ETA: ~10-15 minutes with full optimization\n')

SEEDS = [42, 43, 44, 45, 46]  # Increased seeds for stability and performance
results_df, best = run_training_extreme(seeds=SEEDS, target_auc=0.93)

print('\n' + '='*70)
print('📊 RESULTS SUMMARY')
print('='*70)
display(results_df)

print(f'\n🏆 BEST RESULT:')
print(f'   Seed: {best["seed"]}')
print(f'   L2 AUC: {best["auc_l2"]:.5f}')
print(f'   L3 AUC: {best["auc_l3"]:.5f}')
print(f'   Calibrated AUC: {best["auc_cal"]:.5f}')
print(f'   Threshold: {best["best_thr"]["threshold"]:.3f}')
print(f'   F1 Score: {best["best_thr"]["f1"]:.4f}')

if best['auc_cal'] >= 0.93:
    print(f'\n🎉🎉� BREAKTHROUGH ACHIEVED! {best["auc_cal"]:.5f} >= 93% 🎉🎉🎉')
else:
    gap = 0.93 - best['auc_cal']
    print(f'\n📍 Gap to 93%: {gap:.5f} ({gap*100:.3f} pp)')
    print('💡 Next steps: Add more seeds, try neural blend, or ensemble with other models')

🎯 TARGET: Break 93% AUC barrier
📈 Strategy: L1→L2→L3 stacking + pseudo-labeling + calibration
⏱️  ETA: ~10-15 minutes with full optimization


🚀 SEED 42 (1/5) — Targeting 93%+ AUC

[L1] Training base models...
🚀 Training 5 L1 base models: ['xgb', 'lgb', 'cb', 'rf', 'mlp']
🚀 Training 5 L1 base models: ['xgb', 'lgb', 'cb', 'rf', 'mlp']
  ✓ xgb: [0.92197 0.92046 0.92024 0.91901 0.9185  0.92053 0.91986] → mean 0.92008
  ✓ xgb: [0.92197 0.92046 0.92024 0.91901 0.9185  0.92053 0.91986] → mean 0.92008
  ✓ lgb: [0.92272 0.9223  0.92116 0.92017 0.91999 0.92162 0.92055] → mean 0.92122
  ✓ cb: [0.92051 0.91938 0.91914 0.91812 0.91734 0.91967 0.91886] → mean 0.91900
  ✓ rf: [0.91535 0.91403 0.91379 0.91348 0.9125  0.91465 0.91347] → mean 0.91390
  ✓ mlp: [0.91423 0.91269 0.91005 0.9117  0.90823 0.91313 0.91236] → mean 0.91177

[L2] Training fast linear meta...
  📊 L2 Hill Climbing on 5 base model predictions
  ✓ Hill Climbing optimized weights: [0.2328 0.3771 0.2633 0.     0.1269]
  ✓ L2 Hill Clim

,seed,auc_l2,auc_l3,auc_cal,best_thr
0,42,0.920604,0.920597,0.920708,"{'threshold': 0.44999999999999996, 'f1': 0.943..."
1,43,0.921109,0.921105,0.921208,"{'threshold': 0.49999999999999994, 'f1': 0.943..."
2,44,0.920370,0.920363,0.920471,"{'threshold': 0.49999999999999994, 'f1': 0.943..."
3,45,0.920950,0.920943,0.921042,"{'threshold': 0.49999999999999994, 'f1': 0.942..."
4,46,0.920105,0.920104,0.920209,"{'threshold': 0.44999999999999996, 'f1': 0.942..."



🏆 BEST RESULT:
   Seed: 43
   L2 AUC: 0.92111
   L3 AUC: 0.92110
   Calibrated AUC: 0.92121
   Threshold: 0.500
   F1 Score: 0.9430

📍 Gap to 93%: 0.00879 (0.879 pp)
💡 Next steps: Add more seeds, try neural blend, or ensemble with other models


In [94]:
# 🏅 Build WINNING submission

if 'test' in globals() and test is not None and SAMPLE_SUB_PATH.exists() and best is not None:
    sub = pd.read_csv(SAMPLE_SUB_PATH)
    sub_id_col = sub.columns[0]
    sub_target_col = sub.columns[1] if len(sub.columns) > 1 else (TARGET if TARGET is not None else 'target')
    
    if 'ID_COL' in globals() and ID_COL and sub_id_col != ID_COL and ID_COL in test.columns:
        sub[sub_id_col] = test[ID_COL].values
    
    preds = best['test_cal'] if best.get('test_cal') is not None else None
    
    if preds is not None:
        sub[sub_target_col] = preds
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        auc_str = f"{best['auc_cal']:.5f}".replace('.', '')
        out_path = SUB_DIR / f'EXTREME_93pct_AUC{auc_str}_{timestamp}.csv'
        sub.to_csv(out_path, index=False)
        
        print(f'\n🏆 SUBMISSION SAVED!')
        print(f'   File: {out_path.name}')
        print(f'   AUC: {best["auc_cal"]:.5f}')
        print(f'   Threshold: {best["best_thr"]["threshold"]:.3f}')
        print(f'   Samples: {len(sub):,}')
        
        if best['auc_cal'] >= 0.93:
            print(f'\n🎊 FIRST TO BREAK 93%! Submit this ASAP! 🎊')
    else:
        print('⚠️  No test predictions available.')
else:
    print('⚠️  Submission not created (missing test data or best result).')


🏆 SUBMISSION SAVED!
   File: EXTREME_93pct_AUC092121_20251120_011834.csv
   AUC: 0.92121
   Threshold: 0.500
   Samples: 254,569


Stuff we tryed so far 

. Architecture & Model Stacking
L1 Base Layer: Started with XGBoost, LightGBM, and CatBoost.
Tried: Adding Logistic Regression for diversity (Removed later due to complexity/low gain).
Current State: XGB, LGB, CB with 1500 estimators and learning_rate=0.015.
L2 Meta Layer:
Tried: Fast Linear Stacking (Logistic/Ridge) to save time. Result: Performance dropped (0.917).
Restored: XGBoost Meta-Learner. We switched back to a shallow XGBoost (Depth 4) to capture non-linearities between model predictions.
L3 Pseudo-Labeling:
Implemented: Using high-confidence test predictions (top/bottom 10%) to retrain the ensemble.
Calibration:
Implemented: Isotonic Regression to recalibrate probabilities before the final submission.
2. Feature Engineering
Target Encoding: Applied smoothed target encoding to all categorical features (10-fold CV to prevent leakage).
Interaction Pairs: Created standard ratios and products between key columns (Loan Amount, Income, etc.).
Visual Insight Fixes:
Ordinal Encoding: Mapped grade_subgrade explicitly (A1=1 ... F5=35) because the plots showed a perfect monotonic trend.
Unemployment Flag: Created is_unemployed binary feature based on the specific drop-off seen in the plots.
Log Transforms: Applied np.log1p to Income, Loan Amount, and DTI to handle the massive right-skew seen in histograms.
Risk Features:
Risk Ratio: Added Interest Rate / Credit Score (High rate + Low score = Extreme risk).
PTI Proxy: (Proposed) (Loan * Rate) / Income to approximate payment burden.
3. Hyperparameter Tuning & Correction
Class Weights (The Failed Experiment):
Action: We enabled scale_pos_weight and is_unbalance to handle the 80/20 split.
Result: Score dropped to 0.917. Class weights distorted the ranking probabilities required for AUC.
Fix: Removed all class weights to let the model learn the natural probability distribution.
Estimator Count:
Action: Reduced to 800 for speed.
Correction: Bumped back to 1500 because the model was underfitting the complex feature set.
4. Current Status
Best Score: 0.92020 (Leaderboard).
Current Issue: Generalization Gap. Local CV is higher (~0.9215) than LB (0.9202). The boosting models are correlated and slightly overfitting.
Immediate Next Step: Introduce Bagging (Random Forest) to the L1 layer to reduce variance and close the gap to 0.93.